# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking  current working directory
print(os.getcwd())

# Get  current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    path_list.append(file_path_list)

path_list = path_list[0]
path_list

/home/workspace


['/home/workspace/event_data/2018-11-30-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-11-events.csv',
 '/home/workspace/event_data/2018-11-14-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-28-events.csv',
 '/home/workspace/event_data/2018-11-25-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-12-events.csv',
 '/home/workspace/event_data/2018-11-27-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
# Create a Keyspace sparkify
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)


In [7]:
# Set KEYSPACE to sparkify
session.set_keyspace('sparkify')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# create query 1:  Give me the artist, song title and song's length in the music app history that was heard during |
## sessionId = 338, and itemInSession = 4


# query1 requires session_id as partition key amd iteminsession as clustering key to get a unique primary key

create_query1= """CREATE TABLE IF NOT EXISTS songs_by_sessionid (session_id INT, iteminsession INT, artist_name TEXT, song TEXT, song_length DECIMAL,
            PRIMARY KEY (session_id, iteminsession))"""
try:
    session.execute(create_query1)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 

    for line in csvreader:
        insert_query1 = "INSERT INTO  songs_by_sessionid(session_id, iteminsession, artist_name, song, song_length) "
        insert_query1 = insert_query1 + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(insert_query1, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

In [10]:
# validate query1
select_query1= """SELECT * FROM  songs_by_sessionid WHERE session_id=338 AND iteminsession=4 """
try:
    rows= session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(session_id=338, iteminsession=4, artist_name='Faithless', song='Music Matters (Mark Knight Dub)', song_length=Decimal('495.3073'))


In [11]:
## create query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## query2 requires user_id and session_id as partition keys and iteminsession as clustering key to order by iteminsession
## primary key is made up of two partition key and clusting key(which should not be used in the query)

create_query2= """CREATE TABLE IF NOT EXISTS songs_by_userid (user_id INT, session_id INT,iteminsession INT, artist_name TEXT, song TEXT, first_name TEXT,
            last_name TEXT, PRIMARY KEY ((user_id, session_id),iteminsession))"""
try:
    session.execute(create_query2)
except Exception as e:
    print(e)
                  

                    

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        insert_query2 = "INSERT INTO songs_by_userid(user_id, session_id, iteminsession, artist_name, song, first_name, last_name) "
        insert_query2 = insert_query2 + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(insert_query2,(int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# validate query2
select_query2= """SELECT * FROM songs_by_userid WHERE user_id = 10 AND session_id=182 """
try:
    rows= session.execute(select_query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(user_id=10, session_id=182, iteminsession=0, artist_name='Down To The Bone', first_name='Sylvie', last_name='Cruz', song="Keep On Keepin' On")
Row(user_id=10, session_id=182, iteminsession=1, artist_name='Three Drives', first_name='Sylvie', last_name='Cruz', song='Greece 2000')
Row(user_id=10, session_id=182, iteminsession=2, artist_name='Sebastien Tellier', first_name='Sylvie', last_name='Cruz', song='Kilometer')
Row(user_id=10, session_id=182, iteminsession=3, artist_name='Lonnie Gordon', first_name='Sylvie', last_name='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## query3 requires song as partition key and user_id as clustering key since a single song can be listened by many users
## primary key is made up of one partition key and clusting key (which should not be used in the query)

create_query3= """CREATE TABLE IF NOT EXISTS songs_by_username (user_id INT, song TEXT, first_name TEXT, last_name TEXT, PRIMARY KEY (song, user_id))"""
try:
    session.execute(create_query3)
except Exception as e:
    print(e)
                    

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        insert_query3 = "INSERT INTO songs_by_username(user_id, song, first_name, last_name) "
        insert_query3 = insert_query3 + "VALUES(%s,%s,%s,%s)"
        session.execute(insert_query3,(int(line[10]),line[9], line[1], line[4]))

In [16]:
# validate query3
select_query3= """SELECT * FROM songs_by_username WHERE song= 'All Hands Against His Own'"""
try:
    rows= session.execute(select_query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(song='All Hands Against His Own', user_id=29, first_name='Jacqueline', last_name='Lynch')
Row(song='All Hands Against His Own', user_id=80, first_name='Tegan', last_name='Levine')
Row(song='All Hands Against His Own', user_id=95, first_name='Sara', last_name='Johnson')


In [17]:
# Drop all tables
Drop_query1 = "Drop Table songs_by_sessionid"
try:
    rows = session.execute(Drop_query1)
except Exception as e:
    print(e)
Drop_query2 = "Drop Table songs_by_userid"

try:
    rows = session.execute(Drop_query2)
except Exception as e:
    print(e)
Drop_query3 = "Drop Table songs_by_username"

try:
    rows = session.execute(Drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()